In [1]:
%%html
<style>
table {float:left}
</style>

# <center>Practical HPC</center>
<p>
## <center>Robert Bjornson</center> 
<p>
## <center><i>Yale Center for Research Computing</i></center>
<p>
## <center>Apr 2019</center>

## To follow presentation using mybinder.org

In browser, go to www.mybinder.org

Enter github repo:
https://github.com/ycrc/PracticalHPC

Enter notebook file:
PracticalHPC.ipynb

click "launch"

## What is the Yale Center for Research Computing?


- Independent center under the Provost's office
- Created to support your research computing needs
- Focus is on high performance computing and storage
- ~15 staff, including applications specialists and system engineers
- Available to consult with and educate users
- Manage compute clusters and support users
- Located at 160 St. Ronan st, at the corner of Edwards and St. Ronan
- http://research.computing.yale.edu

## Overview
- Understanding the HPC environment
- Understanding your program's behavior
- Improving performance
- Running parallel programs
- Creating parallel programs

# Assumptions
- Have logged in to a cluster and run simple jobs
- understand basics of Slurm
- want to run more effectively and efficiently


# The HPC Environment

## What is HPC?
- running jobs remotely on a compute cluster 
- linux-based
- using more compute resources (cpus/memory) than you have locally
- access to huge shared storage
- access to specialized hardware, e.g. GPUs 
- access to specialized software
- running large job in parallel, or many sequential jobs


![alt text](clusterschematic.png "Title")

# Typical Cluster

![alt text](yalecluster.jpg "Title")

# Typical compute node
![alt text](nx360-M5.jpg "Title")



## Typical Cluster (Yale, or elsewhere)
- Several hundred servers (nodes)
- Each node has
 - 10s of cpus
 - 100s of GB RAM
 - ~TB local hard drive storage /tmp

- Multiple PB of shared storage
- Fast network connecting nodes and networks
- Shared by 100s of users, 1000s of jobs



## Yale Clusters

| Cluster | Users | Role | CPUs |
| ------- | ----- | ---- | ---- |
| Omega | FAS | Highly Parallel/MPI | 6,500 |
| Grace | FAS | General Purpose | 10,000 |
| Farnam | YSM/LS | General Purpose | 5,500 |
| Ruddle | YCGA | Sequence Data | 3,000 |
| Milgram | Psychology | HIPAA Data | 1,600  |



# Cluster Resources
These are managed by slurm, cgroups, and storage quotas
- Nodes and CPUs
- Node Memory (RAM)
- Job Runtime
- Cluster Disk Storage
- GPUs

### Note
- jobs (unrelated) normally share nodes.  Slurm+cgroups makes sure you have
exclusive access to your resources


# Quick Review of Slurm allocations

$ sbatch [options] batch.sh

$ srun --pty [options] bash

option | example | comment
-------|--------|---
-p _partition_ | -p general | partitions(s) to run on
-c _cpus_ | -c 20 | cpus/task on single node
-n _tasks_ | -n 4 | mpi progs only
-N _nodes_ | -N 10 | forces layout, rarely useful
-t _time_  | -t 7-, -t 0-4 | job killed if exceeded
--mem=_mem_ | --mem=16g | ditto
--mem-per-cpu=_mem_ | --mem-per-cpu=8g | ditto
-J _name_ | -J myjob | name job
--gres | --gres=gpu:p100:2 | request gpus
--array _spec_ | --array 1-10 | array job


## Fairshare Scheduling with backfill
- Slurm tracks cpu usage by user and group
- Job priority is determined by amount of recent usage
- Users and Groups with heavy recent usage have lower priority
- Backfill can run short, lower priority jobs
- Bottom line: only ask for what you need


## Nodes/CPUs
- default: 1 node, 1 cpu
- only request multiple cpus and/or nodes if your program can use them
- simply requesting more nodes or cpus will not make things run faster!
- jobs penned into allocated cpus and memory (cgroups)
- understand -c (--cpus-per-task)
     vs -n (--ntasks) vs -N (--nodes)
- only request GPU or bigmem nodes if you need them
- users have limits (e.g. 100 cpus)
- -C _type_ (westmere, haswell, avx2, etc)

## Memory
- default of 5 GB per allocated cpu on our clusters
- strictly enforced; jobs exceeding limit are killed
- users have limits (e.g. 640 GB)
- you can request custom memory per node or core with sbatch or srun:
```
--mem=6g
--mem-per-cpu=6g
```

## Partitions
- Partitions are subsets of nodes on the cluster
- Each partition has different limits and rules
- Default partition (Day/General)
- Interactive 
- bigmem/gpu



## Scavenge Partition
```
$ sbatch -p scavenge
```

- Compute nodes in other partions are available via scavenge partition
- jobs subject to being killed at any time
- separate per user limits apply
- works best for short jobs, dSQ/array jobs, or jobs that checkpoint
- can include gpus
- compare "findidle -p scavenge"


## Finding compute resources
```
sinfo -p general
sinfo -p general -e -t IDLE -o "%P %.5a %c %m %.10l %.6D %.6t %N"
alias findidle='sinfo -e -t IDLE -o "%P %.5a %c %m %.10l %.6D %.6t %N"'
findidle -p general

```
## Graphical Tools

http://overwatch.ycrc.yale.edu/farnam/orwell/

http://overwatch.ycrc.yale.edu/grace/orwell/

## Special Nodes

## Large Memory Nodes
- Compute nodes with 512GB-1.5TB of RAM
- Reserved for applications with large memory needs. Please be considerate.
- Separate slurm partition: bigmem

Typical allocation: 
```
srun/sbatch -p bigmem --mem=1500g ...
```

## GPU Nodes
- Some applications have been ported to GPUs with impressive performance improvement
- Gpu nodes have conventional cpus with multiple cores, and 1-4 GPUs.  
- To use GPUs, you must:
 - request node(s) with GPUs
 - request the type and number of GPUs 
 - have an application that makes use of GPUs

Typical allocation:
```
srun --pty -p gpu -c 8 --gres=gpu:1080ti:4 ... bash
sbatch -p gpu -c 8 --gres=gpu:1080ti:4 ... batch.sh
```



## GPU Nodes continued

You can also scavenge gpu nodes:
```
srun --pty -p scavenge -c 8 --gres=gpu:1080ti:4 ... bash
sbatch -p scavenge -c 8 --gres=gpu:1080ti:4 ... batch.sh
```

Note that partition names, types and number of GPUs vary by cluster.

Type | FP | Clusters
-------|--------|---
1080ti | single | farnam
K80 | double | grace,farnam
P100, V100 | double | grace

## Node Types with different Instruction Sets
- Our clusters are heterogeneous
- Different node types support different instruction sets: avx, avx2, avx64
- Code compiled on one type may not work on others: Illegal Instruction error
- Simple Rule of Thumb, either:
 - compile only on oldest, lowest instruction set (srun -C sandybridge/ivybridge)
 - only run on most advanced (srun -C avx2)

## Tips for allocating resources
- Whole nodes are often hard to get.  
- Consider asking for most, but not all of a node's cpus or memory
- Use task arrays or dSQ, which make good use of "holes"
- Tune your request for the walltime and memory you need
- Consider submitting to general/day AND scavenge

# Your Job's Behavior

## What is "Running Efficiently"?
- All allocated cores are running near 100% - Disk I/O is not dominating
- GPUs (if allocated) are used
- memory requested is adequate and reasonable


## Monitoring your job
```
squeue -u netid -l (your jobs)
squeue -j jobid -l (particular job)
scontrol show job jobid (during job)
sacct -j jobid -l (after job finishes)
seff jobid     (after job finishes)
```
Default output for squeue and sacct is not ideal.  Put in .bashrc:
```
export SACCT_FORMAT="JobID%-20,JobName,User,Partition,NodeList,Elapsed,State,ExitCode,MaxRSS, AllocTRES%32"
export SQUEUE_FORMAT="%.16i %.12P %.12j %.8u %.2t %.12M %.12l %24R %.4D %.4C %m %8b %8f"
```
in PracticalHPC:

```source SLURM.env```



## Checking cpu and memory utilization for running jobs
Using top/htop
- use ```squeue -u netid``` to determine where your job is running
- ssh to your nodes, especially 2nd and on, and run ```top -u netid``` 
    (or ```htop```)
- look at %CPU and RES columns
- should see ~%100 cpu for each allocated core


## Postmortem checking


After run completes, determine actual usage (See format comment about sacct):

```
$ seff jobid

$ sacct -j jobid

$ /usr/bin/time -a cmd ...

```



## Example: Performance
```
cd Examples/Performance
sbatch bwa.sh # runs out of RAM (default 5GB/core)
sbatch --mem=10g bwa.sh # also runs out
sbatch -c 4 # runs ok.  Show htop, scontrol, sacct, 
htop: H combines threads.  F5 shows tree
seff jobid 
sacct -j jobid
scontrol show job jobid
```

# Data


## Storage Quotas
- most quotas are group quotas
- if exceeded, jobs will die trying to write
- to see all quotas for your group:

```
$ groupquota -u netid

```

## File Storage

Location | Purpose | Typical Limit (size/files)| Backed Up?
-------|--------|---|----|
home | scripts, programs, documents, small files | 125GB/500K | Y
project | large data, programs | 1-4TB/5M | N 
scratch60 | temp data | 10TB/5M | purged
pi\__name_ | dedicated | _varies_ | N






## Other storage options
Google Drive
- Unlimited storage at no cost
- Requires EliApps account
- To be able to create team drives email hpc@yale.edu

Storage@Yale
- ITS service
- Mounted on clusters
- replicated for security
- ~$11/month/TB

## File Transfer
- For small transfers: 
 - Linux/Mac/cygwin: scp, rsync
 - Windows: Mobaxterm, winscp
 
 
- For larger transfers, or outside Yale, or collaborators without Yale credentials:
 - Globus


## Globus
- www.globus.org
- web and command line
- very fast and robust
- transfers between _endpoints_
- endpoints: 
 - Yale clusters: yale#{cluster}
 - Lots of other institutions
 - personal endpoint on your own computer
 - google drive (see http://docs.ycrc.yale.edu/data/google-drive/)
- you can create _shares_, accessible to non-Yale researchers via their globus id


<img src="globus.png" alt="Drawing" style="width: 800px;"/>

## Example
```
show transfer between farnam and ruddle
show personal endpoint (bjornson_mac)
show creating share and sharing with jason.ignatius@yale.edu
```

## Globus cli
```
$ module load Globus-CLI
$ globus login
(you'll get a url.  Open that in a browser, authenticate, and copy the code.  Enter the code on command line.)
$ globus endpoint search bjornson_mac
$ globus endpoint search yale#farnam
(set env vars to BM and F)
$ globus ls -l $BM:/
$ globus transfer -r -s size $BM:Data $F:Data
(returns taskid)
$ globus task show taskid
```
https://docs.globus.org/cli/examples

# Performance in code you write
- When possible, use optimized libraries, rather than roll your own
- Python: numpy or pandas instead of nested lists for matrices
- R, Matlab: vectors instead of loops


## Profiling
- Helps you pinpoint where time is spent

 - R: Rprof
 - Python: cProfile, kernprof/line_profiler
 - C: gprof

# Profiling Python Example:

```
def annotate(gf, mf, of):
    ofp=open(of, "w")
    genenames=[]
    geneinfo=[]
    for l in open(gf):
        gn, chrm, strand, start, end = l.strip().split()
        genenames.append(gn)
        geneinfo.append([chrm, strand, start, end])

    for l in open(mf):
        e=l.strip().split()
        genename=e[-1]
        idx=genenames.index(genename)
        print('\t'.join(e + geneinfo[idx]), file=ofp)


```

## Python: cProfile for function level profiling
```
cd Examples/Profiling
module load Python
(get compute node with srun, module load Python)
python -m cProfile -o bad.prof bad.py
(takes a minute or two)

(in python)
import pstats
p=pstats.Stats('bad.prof')
p.sort_stats('time').print_stats()

or 

```

## kernprof for line-by-line
```
module load Python
(decorate functions with @profile)
kernprof -l bad2.py
python -m line_profiler  bad2.py.lprof

```

` ``
rdb9@c13n08 Profiling]$ python -m line_profiler bad2.py.lprof
Timer unit: 1e-06 s

Total time: 114.565 s
File: bad.py
Function: doit at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           @profile
     5                                           def doit(gf, mf, of):
     6         1          134    134.0      0.0      ofp=open(of, "w")
     7         1            1      1.0      0.0      genenames=[]
     8         1            0      0.0      0.0      geneinfo=[]
     9     80923        44421      0.5      0.0      for l in open(gf):
    10     80922       105965      1.3      0.1          gn, chrm, strand, start, end = l.strip().split()
    11     80922        44241      0.5      0.0          genenames.append(gn)
    12     80922        49721      0.6      0.0          geneinfo.append([chrm, strand, start, end])
    13
    14     71521        90557      1.3      0.1      for l in open(mf):
    15     71520       126260      1.8      0.1          genename=l.strip().split()[-1]
    16
    17     71520    110808166   1549.3     96.7          idx=genenames.index(genename)
    18     71520      3295037     46.1      2.9          print(l + str(geneinfo[idx]), file=ofp)

```

# Parallelism

- Sbatch can allocate multiple cores and nodes, but the script runs on one core on one node sequentially.

- _Simply allocating more nodes or cores DOES NOT make jobs faster._

- How do we use multiple cores to increase speed?


- Two classes of parallelism:
 - Lots of independent sequential jobs
 - Single job parallelized (somehow, maybe by someone else?)
 

## First, my "Disclaimer"
- Parallel computing is a complex field with many tools and techniques
- Entire courses: e.g. Yale CS 424/524 "Parallel Programming Techniques" by Dr. Andrew Sherman
- We'll see some simple techniques for easy cases

## Slurm Job Arrays

- Useful when you have many nearly identical, independent jobs to run
- Starts many identical copies of your script, distinguished by $SLURM_ARRAY_TASK_ID

Submit jobs like this:
```
sbatch --array=1-100 script.sh
```
Inside your batch script this environment variable to do something different in each task:
```
./mycommand -i input.${SLURM_ARRAY_TASK_ID} \
    -o output.${SLURM_ARRAY_TASK_ID}
```




## Slurm Job Arrays continued
A few nice features of job arrays:
- only one job to keep track of
- easy to start or cancel entire set
- slurm options, e.g. cpus, memory, time limits, apply to each task, not overall job
- your allocation can grow and shrink as conditions change
- when using scavenge partition, tasks are killed, but job persists


## Array job example
```
cd Examples/Array
sbatch array.sh
```

## dSQ (aka Dead Simple Queue)
- local tool built on job arrays.  Same nice features, but easier to use
- more flexible; tasks can be arbitrarily different 
- reporting and error recovery built in


## Using dSQ


- Create file containing list of commands to run (jobs.txt)
```
prog arg1 arg2 -o job1.out
prog arg1 arg2 -o job2.out
...
```
- Create batch script
```
module load dSQ
dSQ --jobfile jobs.txt [slurm args] > run.sh
```

slurm args can specify partion, timelimit, memory, etc. in the usual way.

- Submit launch script
```
sbatch run.sh
```

For more info, see <http://research.computing.yale.edu/support/hpc/user-guide/dead-simple-queue>



# dSQ Reporting
- When dSQ job is finished, you'll see a file `job_<jobid>_status.tsv`
- Generate report:
```
$ dSQAutopsy jobs.txt job_<jobid>_status.tsv > failedjobs.txt
Autopsy Task Report:
9 succeeded
1 failed
0 didn't run.
```

- If any jobs failed, failedjobs.txt will contain those jobs





## dSQ Example
```
cd Examples/GenomePL
python mktasks.py data output > tasks.sh # create dSQ job file
module load dSQ
dSQ --jobfile tasks.sh > run.sh
sbatch run.sh # will fail, do sacct -j jobid, resubmit --mem=10g
```

# Running already parallelized applications

## Parallel bootstrap in R using Multicore library
- works on multiple cpus on one node

```
boot(data=trees, statistic=volume_estimate, R=50000, parallel="multicore", ncpus=8)


cd Examples/R-Bootstrap
sbatch ... boot.sh
sbatch ... -c 8 parboot.sh
```



## Parallel Namd
- molecular dynamics simulation written in C and C++
- Namd is parallelized several ways, which can be combined:
 - OpenMP (within one node)
 - MPI (across nodes)
 - GPU
 
Example: Satellite Tobacco Mosaic Virus 1,066,628 atoms, 500 time steps


![alt text](Examples\Namd\namd.png "Title")


# Namd Performance on Grace

   |startup (s)|simulate|total|step|step s/u
---|-------|--------|-----|----|----
1 node, 1 cpu|14|5833|5847|11.6|1.0
1 node, 28 cpu|13|218|231|0.44|26.3
1 node, 20 cpu+4 K80 gpus|12|32|44|.056|207
9 nodes, 40 cpus|29.5|212|242|.424|27.4

in /home/fas/lsprog/rdb9/repos/PracticalHPC/Examples/Namd/stmv
- gpus slurm-9139008.out
- mpi slurm-9152709.out
- 28 core MP slurm-9139007.out
- 1 core slurm-9140416.out

## Namd Example
```
cd Examples/Namd/stmv
sbatch multi.sh
sbatch gpu.sh
use htop and nvidia-smi on allocated node
```




## Case Study

```
for d in lung heart liver ...
  for exp in e1 e2 e3 ...
    for p in 0.01 0.05 ...
       for c in 1 2 3 ... 22
          sbatch job.py $d $exp $p bigfile.$c
          

```

Assuming processing chrom c requires a very large input file, what's wrong with this?

## Restructure
Allow all jobs using same c to run on same node, staging input file.  Use array job to handle chromosomes

```
#SBATCH -t 1-22

C=$SLURM_ARRAY_TASK_ID
cp bigfile.$C /tmp

for d in lung heart liver ...
  for exp in e1 e2 e3 ...
    for p in 0.01 0.05 ...
       prog d exp p /tmp/bigfile.$C
       
rm /tmp/bigfile.$C

```


## Parallelize using gnu parallel

https://www.gnu.org/software/parallel


```
#SBATCH -t 1-22
#SBATCH -c 20

module load parallel
C=$SLURM_ARRAY_TASK_ID
cp bigfile.$C /tmp

parallel prog {1} {2} {3} {N} ::: lung heart liver ::: e1 e2 e3 ::: 0.01 0.05
       
rm /tmp/bigfile.$C

```

# DIY Parallelism

## Parallel Kmeans in R using parallel

- simple idea: reduce computation to applying function to list of values (very R-ish)
- then do that in parallel

sequential
```
starts=100000; tasks=8; nstarts=rep(starts/tasks, tasks)
results<-lapply(nstarts, function(nstart) kmeans(Boston, 4, nstart=nstart)))
```

parallel
```
library(parallel)
...
results<-mclapply(nstarts, 
  function(nstart) kmeans(Boston, 4, nstart=nstart), 
  mc.cores=cores)
```





## Example Kmeans
```
cd R-Kmeans
sbatch kmeans.sh
sbatch parkmeans.sh
```



## kmeans performance on farnam m610s

   |runtime|s/u
---|-------|-------
1 node, 1 cpu| 44.475 | -
1 node, 8 cpus | 5.96 | 7.46

## Other thoughts on parallel R
- _Parallel R_, by Steve Weston, O'Reilly Press
- YCRC Bootcamp: Writing Efficient R Code (taught by Steve)
- ```foreach``` parallel construct (written by Steve)

## Python Multiprocessing
- Support for creating parallel processes and communicating and synchronizing
- Simplest interface is parallel map, very similar to R's mclapply

```
import multiprocessing
p=multiprocessing.Pool(10)
inputs=[...]
results=p.map(f, inputs)
```

- Pool creates set of "workers" that compute f() on inputs
- Workers are forked processes.  Have copies of data at point where pool was created
- Function, inputs, and outputs are serialized and passed between master and workers

## Parallel Travelling salesman in Python using multiprocessing
- Given N cities, find shortest route visiting each once
- NP-complete -> very expensive to find optimal solution
- We'll use a heuristic approach, simulated annealing:

```
T = initialT
curr = best = initialsolution
while T > stopT:
    candidate = randomly swap two adjacent cities in curr
    if cost(candidate) < cost(best):
        curr=candidate
        best=candidate
    elif cost(candidate)-cost(best) < T
        curr=candidate
    T = T * alpha
```

## Parallelization strategy
- Run K copies, starting from different initial conditions
- Take different random paths
- Choose the best solution

Example/simulated-annealing-tsp

Steps:
- seq.py: explicit for loop over simulations
- map.py: convert loop to map
- parmap.py: convert map to parallel map



## parmap.py

```
def dotrial(t):
    t.anneal()
    return t

p=multiprocessing.Pool(cores) 
trials = [SimAnneal(coords, ...) for i in range(ntrials)]
got=p.map(dotrial, trials, chunksize=5)
```

## Example: TSP
```
cd Examples/TSP
srun -p interactive --pty bash
python seq.py 10
python map.py 10
srun -c 10 -p interactive --pty bash
python parmap.py 100
```

## Subtleties WRT random numbers
- When running in parallel, what random number sequence is seen by each process?
- many random number generators use system clock, which may not be different for each process
- you may want reproduceable results

- R: use RNGkind("L'Ecuyer-CMRG").  Sets up independent, reproducible random streams
- Matlab and Python: Each worker has same seed by default.  You should explicitly set it, e.g. rng(workerid)


## C/C++: OpenMP
- popular way to parallelize C programs on single node
- annotate for loop with #pragma omp for 

```
#pragma omp for reduction(+:sum)
  for (i=0; i<len*threads; i++)
    {
      sum += (a[i] * b[i]);
      psum = sum;
    }
    ```


## Example: Matrix multiply with OpenMP
```
cd Examples/OpenMP
make mm
sbatch mm.sh
```


## To get help or report problems
- Check our status page: http://research.computing.yale.edu/system-status
- Send an email to our tracking system: hpc@yale.edu
- Read documentation: http://research.computing.yale.edu/hpc-support
- Office hours: http://research.computing.yale.edu/hpc-support/office-hours-support
- Email me directly: Robert.bjornson@yale.edu